In [1]:
#!pip install kaggle
#!mkdir ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!kaggle datasets download -d puneet6060/intel-image-classification
!unzip -o intel-image-classification.zip &> /dev/null


In [2]:
!mkdir seg_val && mv seg_test/seg_test/** seg_val/ && rm -rf seg_test**
!mkdir seg_test && mv seg_pred/seg_pred/** seg_test/ && rm -rf seg_pred**
!mv seg_train/seg_train/** seg_train/ && rm -rf seg_train/seg_train

In [ ]:
import glob
import os

# fileList = glob.glob('seg_**/**/*.jpg', recursive=True)
# for file in fileList:
#     if len(os.path.basename(file)) > 6:
#         os.remove(file)

# folders = ['seg_train', 'seg_val']
# for folder in folders:
#     for dir in os.listdir(folder):
#         i = 0
#         files = sorted(os.listdir(os.path.join(folder, dir)))
#         for file in sorted(os.listdir(os.path.join(folder, dir))):
#             if i > 10 and len(files) > 20:
#                 os.remove(os.path.join(os.path.join(folder, dir), file))
#             i += 1

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from layers.Activation import Activation, SoftmaxActivation, ReluActivation
from layers.Conv2D import Conv2D
from layers.ConvNeuralNetwork import NeuralNetwork
from layers.Dense import Dense
from layers.Flatten import Flatten
from layers.Pooling import MaxPooling2D
from layers.utils import AdamOptimizer, CrossEntropy

In [ ]:
train_dir = './seg_train'
val_dir = './seg_val'

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
images = [train_ds.take(1)]
images[0]

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
rescale = tf.keras.layers.Rescaling(1. / 255)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.map(lambda x, y: (rescale(x), y))

In [ ]:
image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
print("Min and max values after rescaling:", np.min(first_image), np.max(first_image))

In [ ]:
def get_processed_input(dataset):
    X = []
    y = []
    for image_batch, label_batch in dataset:
        for i in range(BATCH_SIZE):
            if i < image_batch.shape[0]:
                X.append(image_batch[i].numpy())
                y.append(label_batch[i].numpy())

    X = np.array(X)
    y = np.array(y)

    X = np.moveaxis(X, -1, 1)
    y = to_categorical(y.astype("int"))

    return X, y


X_train, y_train = get_processed_input(train_ds)
X_val, y_val = get_processed_input(train_ds)

In [ ]:
MAX_SIZE = 50
X_train = np.array(X_train)[:MAX_SIZE]
y_train = np.array(y_train)[:MAX_SIZE]
print("Shape of X_train, y_train:", X_train.shape, y_train.shape)


In [ ]:
class DeepLearningModel:

    def __init__(self, n_inputs, n_outputs, val_data):
        model = NeuralNetwork(optimizer=AdamOptimizer(), loss=CrossEntropy, val_data=val_data)
        model.add(Conv2D(input_shape=n_inputs, n_filters=16, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2), stride=2, padding='same'))

        model.add(Conv2D(n_filters=32, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Conv2D(n_filters=64, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Conv2D(n_filters=128, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Flatten())
        model.add(Dense(256))
        # model.add(BatchNormalization())
        model.add(Activation(ReluActivation))

        model.add(Dense(256))
        # model.add(BatchNormalization())
        model.add(Activation(ReluActivation))

        model.add(Dense(n_outputs))
        model.add(Activation(SoftmaxActivation))

        self.model = model

    def get_model(self):
        return self.model

In [ ]:

model = DeepLearningModel(n_inputs=(3, 154, 154), n_outputs=6, val_data=(X_val, y_val)).get_model()
model.summary()

In [ ]:

train_err, val_err = model.fit(X_train, y_train, n_epochs=5, batch_size=BATCH_SIZE)

In [ ]:
# Training and validation error plot
n = len(train_err)
training, = plt.plot(range(n), train_err, label="Training Error")
validation, = plt.plot(range(n), val_err, label="Validation Error")
plt.legend(handles=[training, validation])
plt.title("Error Plot")
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.show()


In [15]:
MAX_SIZE = 50
X_train = np.array(X_train)[:MAX_SIZE]
y_train = np.array(y_train)[:MAX_SIZE]
print("Shape of X_train, y_train:", X_train.shape, y_train.shape)


Shape of X_train, y_train: (500, 3, 224, 224) (500, 6)


In [18]:
class DeepLearningModel:

    def __init__(self, n_inputs, n_outputs, val_data):
        model = NeuralNetwork(optimizer=AdamOptimizer(), loss=CrossEntropy, val_data=val_data)
        model.add(Conv2D(input_shape=n_inputs, n_filters=16, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2), stride=2, padding='same'))

        model.add(Conv2D(n_filters=32, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Conv2D(n_filters=64, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Conv2D(n_filters=128, filter_shape=(2, 2), stride=1, padding='same'))
        # model.add(BatchNormalization(axis=0))
        model.add(Activation(ReluActivation))
        model.add(MaxPooling2D(pool_shape=(2, 2)))  # Valid padding

        model.add(Flatten())
        model.add(Dense(256))
        # model.add(BatchNormalization())
        model.add(Activation(ReluActivation))

        model.add(Dense(256))
        # model.add(BatchNormalization())
        model.add(Activation(ReluActivation))

        model.add(Dense(n_outputs))
        model.add(Activation(SoftmaxActivation))

        self.model = model

    def get_model(self):
        return self.model

In [ ]:

model = DeepLearningModel(n_inputs=(3, 154, 154), n_outputs=6, val_data=(X_val, y_val)).get_model()
model.summary()

In [ ]:

train_err, val_err = model.fit(X_train, y_train, n_epochs=5, batch_size=BATCH_SIZE)

In [ ]:
# Training and validation error plot
n = len(train_err)
training, = plt.plot(range(n), train_err, label="Training Error")
validation, = plt.plot(range(n), val_err, label="Validation Error")
plt.legend(handles=[training, validation])
plt.title("Error Plot")
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.show()
